Radosław Jadach - projekt zaliczeniowy Bootcamp Data Science - cześć 2, modelowanie

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_predict, StratifiedKFold, cross_val_score
from sklearn import model_selection

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as ss
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import timeit

In [2]:
data=pd.read_csv('data_clean')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
data.shape

(87020, 33)

In [4]:
data.head(10)

,Unnamed: 0,Gender,City,Monthly_Income,Loan_Amount_Applied,Loan_Tenure_Applied,Existing_EMI,Employer_Name,Mobile_Verified,Var5,Var1,Loan_Amount_Submitted,Loan_Tenure_Submitted,Interest_Rate,Processing_Fee,EMI_Loan_Submitted,Filled_Form,Device_Type,Var2,Source,Var4,Disbursed,Age,Salary_Account_Bank of Baroda,Salary_Account_Bank of India,Salary_Account_Citibank,Salary_Account_HDFC Bank,Salary_Account_ICICI Bank,Salary_Account_IDBI Bank,Salary_Account_Kotak Bank,Salary_Account_Pozostałe,Salary_Account_Punjab National Bank,Salary_Account_State Bank of India
0,0,1,11,27.144176,547.722558,5.0,0.000000,1,0,0,8,0.000000,0.0,0,0.000000,0,0,1,5,7,1,0,3.610918,0,0,0,1,0,0,0,0,0,0
1,1,0,9,32.710663,447.213595,2.0,0.000000,10,1,13,4,447.213595,2.0,1,0.000000,1,0,1,5,7,3,0,3.401197,0,0,0,0,1,0,0,0,0,0
2,2,0,0,28.231081,774.596669,4.0,0.000000,1,1,0,8,670.820393,4.0,0,0.000000,1,0,1,6,4,1,0,3.526361,0,0,0,0,0,0,0,0,0,1
3,3,0,0,32.710663,1000.000000,5.0,0.000000,1,1,10,8,959.166305,5.0,0,0.000000,1,0,1,6,4,3,0,3.332205,0,0,0,0,0,0,0,0,0,1
4,4,0,10,46.415888,707.106781,2.0,10.126671,1,1,17,8,707.106781,2.0,0,0.000000,1,0,1,6,1,3,0,3.433987,0,0,0,1,0,0,0,0,0,0
5,5,0,10,35.568933,547.722558,5.0,9.615872,5,1,17,0,547.722558,5.0,1,7.313887,1,0,1,6,4,3,0,3.496508,0,0,0,0,0,0,0,1,0,0
6,6,1,0,41.212853,2.449490,5.0,0.000000,1,0,0,8,0.000000,0.0,0,0.000000,0,0,1,6,6,1,0,3.332205,0,0,0,0,0,0,0,1,0,0
7,7,0,10,27.144176,447.213595,5.0,7.862497,1,1,3,8,447.213595,5.0,0,0.000000,1,0,1,6,5,3,0,3.688879,0,0,0,0,0,0,0,1,0,0
8,8,0,0,42.171633,0.000000,0.0,0.000000,1,1,13,3,1140.175425,5.0,1,10.165890,1,1,0,4,7,5,0,3.761200,0,0,0,0,0,0,0,0,0,1
9,9,1,9,31.072325,547.722558,3.0,0.000000,1,1,0,7,547.722558,3.0,1,7.313887,1,0,1,6,6,1,0,3.258097,0,0,0,0,0,0,1,0,0,0


In [5]:
# Usuwamy Unnamed (index)
data=data.drop('Unnamed: 0',axis=1)

## Zmienna wynikowa

In [6]:
data['Disbursed'].value_counts()

0    85747
1     1273
Name: Disbursed, dtype: int64

In [ ]:
#Widzimy duże niezbalansowanie - konieczne będzie zbalansowanie klas!
# Dzielimy zbiór

In [7]:
X = data.drop('Disbursed',axis=1, inplace=False).values
X.shape

(87020, 31)

In [8]:
y = data['Disbursed']
y.shape

(87020,)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
# Sprawdzamy jak rozłożyły się próbki
y_train.mean(), y_test.mean()

(0.0143415306826017, 0.015490691794989658)

# BASELINE

In [11]:
# Z treści zadania wynika, że interesuje nas jak największa precyzja w trafianiu 1. Czyli naszą miarą powinno być recall.
# Jednocześnie musimy uważać, żeby model nie przewidywał wszystkich jako 1 :)

In [12]:
kf = model_selection.StratifiedKFold(n_splits=5,
                                     shuffle=False)

In [13]:
rf = RandomForestClassifier()

cvs = cross_val_score(rf, X_train, y_train, cv=kf, scoring='recall')
cvs

array([0.        , 0.00534759, 0.00534759, 0.        , 0.        ])

In [ ]:
# Bardzo źle to wygląda - spróbujmy zmienić parametry i zastosować oversampling

In [16]:
%%time
pipe = Pipeline([
        ('sampling', SMOTE()),
        ('rfc', RandomForestClassifier(verbose = True))
    ])

params = {
    'rfc__max_depth': [2, 5, 10, 15, 25, 50, 75, 100, 150],
    'rfc__min_samples_leaf': [1, 2, 5, 10, 20, 50]
}

baseline = GridSearchCV(estimator = pipe,
                          param_grid=params,
                          scoring = 'recall',
                          verbose = True,
                          n_jobs = -1,
                          cv = kf
                          )

baseline.fit(X_train, y_train)
baseline.best_score_, baseline.best_estimator_

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Wall time: 39min 38s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    7.5s finished


(0.7456934804869724,
 Pipeline(steps=[('sampling', SMOTE()),
                 ('rfc',
                  RandomForestClassifier(max_depth=2, min_samples_leaf=10,
                                         verbose=True))]),
 {'rfc__max_depth': 2, 'rfc__min_samples_leaf': 10})

In [17]:
baseline.cv_results_['mean_test_score']

array([0.71581522, 0.72548071, 0.72007623, 0.74569348, 0.71368188,
       0.72642508, 0.61858573, 0.61859711, 0.60682103, 0.60578564,
       0.61110479, 0.59934577, 0.24252475, 0.24466378, 0.24252475,
       0.23609626, 0.24360564, 0.26175333, 0.07052566, 0.06945614,
       0.08333713, 0.0833485 , 0.10256571, 0.13782   , 0.01602003,
       0.01281716, 0.01709523, 0.02885425, 0.04808283, 0.09296279,
       0.01174764, 0.01495051, 0.01815906, 0.02992377, 0.04916942,
       0.0983047 , 0.01174764, 0.01388668, 0.01496188, 0.02885994,
       0.05130845, 0.1036466 , 0.01174764, 0.01281716, 0.01602003,
       0.02565138, 0.05663329, 0.1036466 , 0.01388668, 0.01067812,
       0.01816475, 0.02885425, 0.04702469, 0.10149619])

In [ ]:
# Uzyskane wyniki są znacząco wyższe. 
# Jednocześnie widzimy bardzo duży rozrzut w zależności od zadanych parametrów
# Najlepszy ze wskazanych, tj. 74,5% będzie naszym progiem odniesienia

# MODELE

In [ ]:
# 1. Parametry do modeli zadawane siatką lub wybierane losowo:
#     a) dla siatki, jeżeli otrzymane wartości będą na skraju przedziału, poszerzamy go
#     b) liczba iteracji dla wyznaczana ekspercko - wypadkowa szerokości przedziału, czasochłonności i ilości parametrów
# 2. Z powodu niezbalansowania klas stosujemy zarówno under- jak i oversampling (na wartościach domyśnych)

In [19]:
%%time
model = Pipeline([
        ('over', SMOTE()),
        ('under', RandomUnderSampler()),
        ('scaler', StandardScaler()),
        ('estimator', LogisticRegression(max_iter = 1000, verbose=2))
    ])

params = {
    'estimator__C': ss.uniform(0.1, 200)
}

rscv_1 = RandomizedSearchCV(estimator = model,
                            param_distributions=params,
                            scoring = 'recall',
                            n_iter = 100,
                            verbose = 2,
                            n_jobs = -1,
                            cv = kf
                         )
    
rscv_1.fit(X_train, y_train);
    
rscv_1.best_params_, rscv_1.best_score_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Wall time: 4min 20s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished


({'estimator__C': 117.23293078738999}, 0.7564114233701217)

In [20]:
%%time
model = Pipeline([
        ('sampling', SMOTE()),
        ('under', RandomUnderSampler()),
        ('tree', DecisionTreeClassifier())
    ])

params = {
    'tree__max_depth': ss.randint(1, 100),
    'tree__min_samples_leaf': ss.randint(1, 150)
}

rscv_2 = RandomizedSearchCV(estimator = model,
                            param_distributions=params,
                            scoring = 'recall',
                            n_iter = 50,
                            verbose = 1,
                            n_jobs = -1,
                            cv = kf
                      )

rscv_2.fit(X_train, y_train)

rscv_2.best_estimator_, rscv_2.best_score_

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Wall time: 3min 32s


(Pipeline(steps=[('sampling', SMOTE()), ('under', RandomUnderSampler()),
                 ('tree',
                  DecisionTreeClassifier(max_depth=1, min_samples_leaf=6))]),
 0.8781772670383432)

In [21]:
%%time
model = Pipeline([
        ('sampling', SMOTE()),
        ('under', RandomUnderSampler()),
        ('KNeighbors', KNeighborsClassifier())
    ])

params = {
    'KNeighbors__n_neighbors': [1, 2, 5, 10, 15, 25, 50, 100, 125, 150, 175]
}

grid_3 = GridSearchCV(estimator = model,
                     param_grid = params,
                     scoring = 'recall',
                     verbose = True,
                     n_jobs = -1,
                     cv = kf)

grid_3.fit(X_train, y_train)

grid_3.best_params_, grid_3.best_score_

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Wall time: 21min 53s


({'KNeighbors__n_neighbors': 175}, 0.7275571737399021)

In [22]:
%%time
model = Pipeline([
        ('sampling', SMOTE()),
        ('under', RandomUnderSampler()),
        ('scaler', StandardScaler()),
        ('xgb', XGBClassifier(verbosity=1))
    ])

params = {
    'xgb__learning_rate': ss.uniform(0.001, 0.6),
    'xgb__max_depth': ss.randint(3, 20),
    'xgb__min_child_weight': ss.randint(3, 15),
    'xgb__reg_lambda': ss.uniform(0.01, 5),
    'xgb__reg_alpha': ss.uniform(0.01,5),
    'xgb__gamma': ss.uniform(0.01, 5)
}

rscv_4 = RandomizedSearchCV(estimator = model,
                            param_distributions=params,
                            scoring = 'recall',
                            n_iter = 50,
                            verbose = 1,
                            n_jobs = -1,
                            cv = kf
                         )

rscv_4.fit(X_train, y_train)

rscv_4.best_params_, rscv_4.best_score_

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Wall time: 1h 31min 57s


({'xgb__gamma': 2.8558039122163406,
  'xgb__learning_rate': 0.009162987974344314,
  'xgb__max_depth': 5,
  'xgb__min_child_weight': 14,
  'xgb__reg_alpha': 1.6050440552075207,
  'xgb__reg_lambda': 3.5971773874158615},
 0.5085106382978724)

In [23]:
best_models = []
best_models.append(rscv_1.best_estimator_) # Regresja
best_models.append(rscv_2.best_estimator_) # drzewo
best_models.append(grid_3.best_estimator_) # sasiedzi
best_models.append(rscv_4.best_estimator_) # XGB

accuracy_train = []
auc_train = []
precision_train = []
recall_train = []
f1_train = []
tn_train = []
fp_train = []
fn_train = []
tp_train = []
fpr_train = []
tpr_train = []

accuracy_test = []
auc_test = []
precision_test = []
recall_test = []
f1_test = []
tn_test = []
fp_test = []
fn_test = []
tp_test = []
fpr_test = []
tpr_test = []

In [24]:
%%time

# uzupełniamy listy
for model in best_models:
    y_train_predict = model.predict(X_train)
    y_test_predict = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_true = y_train, y_pred =y_train_predict).ravel()
    tn_t, fp_t, fn_t, tp_t = confusion_matrix(y_true = y_test, y_pred = y_test_predict).ravel()
    
    
    accuracy_train.append(accuracy_score(y_train, y_train_predict))
    auc_train.append(roc_auc_score(y_train, y_train_predict))
    precision_train.append(precision_score(y_true = y_train, y_pred=y_train_predict))
    recall_train.append(recall_score(y_true = y_train, y_pred=y_train_predict))
    f1_train.append(f1_score(y_train, y_train_predict))
    tn_train.append(tn)
    fp_train.append(fp)
    fn_train.append(fn)
    tp_train.append(tp)
    fpr_train.append(fp / (fp + tn))
    tpr_train.append(tp / (tp + fn))
    
    
    accuracy_test.append(accuracy_score(y_test, y_test_predict))
    auc_test.append(roc_auc_score(y_test, y_test_predict))
    precision_test.append(precision_score(y_true = y_test, y_pred=y_test_predict))
    recall_test.append(recall_score(y_true = y_test, y_pred=y_test_predict))
    f1_test.append(f1_score(y_test, y_test_predict))
    tn_test.append(tn_t)
    fp_test.append(fp_t)
    fn_test.append(fn_t)
    tp_test.append(tp_t)
    fpr_test.append(fp_t / (fp_t + tn_t))
    tpr_test.append(tp_t / (tp_t + fn_t))

Wall time: 5min 18s


In [25]:
summary_train = {
     'Accuracy_train': accuracy_train,
     'AUC_train': auc_train,
     'Precision_train': precision_train, 
     'Recall_train': recall_train,
     'F1_train': f1_train,
     'TN_train': tn_train, 
     'FP_train': fp_train,
     'FN_train': fn_train,
     'TP_train': tp_train,
     'FPR_train': fpr_train,
     'TPR_train': tpr_train
    }

wyniki_train = pd.DataFrame(data=summary_train)
wyniki_train.insert(loc=0, column='Method', value=['Logistic_Regression',
                                             'DecisionTreeClassifier',
                                             'KNeighborsClassifier',
                                             'XGBClassifier'
                                             ])
wyniki_train

,Method,Accuracy_train,AUC_train,Precision_train,Recall_train,F1_train,TN_train,FP_train,FN_train,TP_train,FPR_train,TPR_train
0,Logistic_Regression,0.747368,0.759720,0.042534,0.772436,0.080629,48054,16275,213,723,0.252996,0.772436
1,DecisionTreeClassifier,0.585260,0.729601,0.029594,0.878205,0.057258,37375,26954,114,822,0.419002,0.878205
2,KNeighborsClassifier,0.697954,0.792033,0.040703,0.888889,0.077841,44720,19609,104,832,0.304824,0.888889
3,XGBClassifier,0.793427,0.692541,0.040372,0.588675,0.075562,51232,13097,385,551,0.203594,0.588675


Wszystkie klasyfikatory, za wyjątkiem XGB, otrzymały wynik lepszy niż baseline.

Najwyższą wartość recall w treningu otrzymaliśmy  dla klasyfikatora KNeighbors. Jednocześnie widzimy bardzo wysoką wartość FP. W przypadku udzielenia pożyczki osobie, która nie powinna jej dostać jest to doć poważny problem.

Druga w kolejności wartość parametru recall, otrzymana przez drzewo, jest nieznacznie niższa. Jednak zarówno wartości accuracy jak i FP są niższe niż dla klasyfikatora KNeghbors. Zachodzi podejrzenie przetrenowania modelu.

Skupmy się zatem na dwóch pozostałych, tj. regresji  XGB:
    
    a) regresja - gorsze accuracy, porównywalna precyzja, ISTOTNIE wyższa wartość recall przy nieznacznie wyższej wartości f1
    
    b) XGB - lepsza precyzja, ISTOTNIE gorszy recall
    
Wydaje się zatem, że bazując na zadanym na wstępie parametrze należałoby odrzucić XGB. Sprawdzamy jeszcze wyniki na zbiorze walidacyjnym:

In [26]:
summary_test = {
     'Accuracy_test': accuracy_test,
     'AUC_test': auc_test,
     'Precision_test': precision_test, 
     'Recall_test': recall_test, 
     'F1_test': f1_test,
     'TN_test': tn_test, 
     'FP_test': fp_test, 
     'FN_test': fn_test, 
     'TP_test': tp_test,
     'FPR_est': fpr_test,
     'TPR_test': tpr_test
    }

wyniki_test = pd.DataFrame(data=summary_test)
wyniki_test.insert(loc=0, column='Method', value=['Logistic_Regression',
                                             'DecisionTreeClassifier',
                                             'KNeighborsClassifier',
                                             'XGBClassifier'
                                             ])
wyniki_test

,Method,Accuracy_test,AUC_test,Precision_test,Recall_test,F1_test,TN_test,FP_test,FN_test,TP_test,FPR_est,TPR_test
0,Logistic_Regression,0.750908,0.753747,0.045601,0.756677,0.086018,16081,5337,82,255,0.249183,0.756677
1,DecisionTreeClassifier,0.583406,0.709568,0.030456,0.839763,0.058781,12409,9009,54,283,0.420628,0.839763
2,KNeighborsClassifier,0.695334,0.728444,0.037766,0.762611,0.071969,14870,6548,80,257,0.305724,0.762611
3,XGBClassifier,0.790071,0.680175,0.041414,0.566766,0.077187,16997,4421,146,191,0.206415,0.566766


W przypadku testów na zbiorach walidacyjnych znów recall jest niższe tylko dla XGB. 
Celem zadania było "to identify the segment of customers having higher disbursal rate in next 30 days."

Wydaje się więc że powinniśmy zdecydować się na wybór drzewa. Patrząc jednak na uzyskane wyniki, w szczególności wartość F1, FP, TP zdecydowałbym się na regresję.
W przypadku drzewa zachodzi podejrzenie przeuczenia - mamy największą ilość FP (2x więcej niż XGB i 1,7x więcej niż regresja - świetnie widać to patrząc na wartości F1)

W treści zadania jest mowa o reklamach dla potencjalnych klientów (i potem konwersji) - uwzgledniając ten fakt zdecydowałbym się, mimo wszystko, na **XGB**.
Wziąłbym tu pod uwagę kwestie wizerunkowe (prócz czystych danych) - lepiej wyświetlić niechcianą reklamę 4,4k klientów (FP XGB) niż 5,4k (regresja)